## 한국수출입은행 - 원화 환율(최근까지) 및 DB 저장
+ 작성: 임경호
+ 한국수출입은행 Open API https://www.koreaexim.go.kr/ir/HPHKIR019M01

### 1. 데이터를 가져올 날짜 설정
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [1]:
import requests
import pandas as pd

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [2]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date FROM fx_eximbank ORDER BY date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
date_from = myutil.get_next_day(last_date)
date_to = myutil.get_previous_day()

### 2. API를 활용한 데이터 수집 (한국수출입은행 Open API)

In [3]:
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

In [4]:
df_fx_all.shape

(69, 11)

In [5]:
df_fx_all

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,20230724,AED,345.46,352.43,348.95,348,0,0,348,348.95,아랍에미리트 디르함
1,20230724,AUD,853.44,870.69,862.07,862,0,0,862,862.07,호주 달러
2,20230724,BHD,"3,365.82","3,433.81","3,399.82","3,399",0,0,"3,399","3,399.82",바레인 디나르
3,20230724,BND,953.54,972.81,963.18,963,0,0,963,963.18,브루나이 달러
4,20230724,CAD,959.63,979.02,969.33,969,0,0,969,969.33,캐나다 달러
...,...,...,...,...,...,...,...,...,...,...,...
64,20230726,SAR,338.05,344.88,341.47,341,0,0,341,341.47,사우디 리얄
65,20230726,SEK,122.26,124.73,123.5,123,0,0,123,123.5,스웨덴 크로나
66,20230726,SGD,954.91,974.2,964.56,964,0,0,964,964.56,싱가포르 달러
67,20230726,THB,36.83,37.58,37.21,37,0,0,37,37.21,태국 바트


### 3. 데이터 전처리

In [6]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

In [7]:
# 특정 통화 환율 확인
df_fx_all_save[df_fx_all_save['cur_unit'].str.contains('USD')]

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
22,20230724,USD,"1,268.88","1,294.51","1,281.7","1,281","1,281","1,281.7"
45,20230725,USD,"1,270.96","1,296.63","1,283.8","1,283","1,283","1,283.8"
68,20230726,USD,"1,267.89","1,293.5","1,280.7","1,280","1,280","1,280.7"


In [8]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [9]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230724,AED,345.46,352.43,348.95,348,348,348.95
1,20230724,AUD,853.44,870.69,862.07,862,862,862.07
2,20230724,BHD,3365.82,3433.81,3399.82,3399,3399,3399.82
3,20230724,BND,953.54,972.81,963.18,963,963,963.18
4,20230724,CAD,959.63,979.02,969.33,969,969,969.33
...,...,...,...,...,...,...,...,...
64,20230726,SAR,338.05,344.88,341.47,341,341,341.47
65,20230726,SEK,122.26,124.73,123.5,123,123,123.5
66,20230726,SGD,954.91,974.2,964.56,964,964,964.56
67,20230726,THB,36.83,37.58,37.21,37,37,37.21


In [10]:
# 특정 통화 환율 확인
df_fx_all_save[df_fx_all_save['cur_code'].str.contains('USD')]

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
22,20230724,USD,1268.88,1294.51,1281.7,1281,1281,1281.7
45,20230725,USD,1270.96,1296.63,1283.8,1283,1283,1283.8
68,20230726,USD,1267.89,1293.5,1280.7,1280,1280,1280.7


### 4. 테이블에 데이터 저장 (mdw.weather)

In [11]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()